In [2]:
import numpy as np
import pandas as pd 
import torch

import re

In [3]:
combined = pd.read_csv("data/reddit/train.csv", delimiter=",")

In [4]:
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

Using cache found in /home/maha.agro/.cache/torch/hub/huggingface_pytorch-transformers_main


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased')
model = model.to(device)

Using cache found in /home/maha.agro/.cache/torch/hub/huggingface_pytorch-transformers_main
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def getFeatures(batch_1):

  # lambda function to tokenize the input batch 
  # output length 512 embeddings 
  processed = batch_1[0].apply(lambda x: re.sub(r'http\S+', '', x))
  processed = processed.apply(lambda x: re.sub(r"[\([{})\]]", "", x))
  tokenized = processed.apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512)))
  # The tokenizers output contains the 
  # input id
  # the token_type_ids 
  # and the attention mask 
  max_len = 50
  for i in tokenized.values:
    if len(i) > max_len:
      max_len = len(i)
    
  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

  attention_mask = np.where(padded != 0, 1, 0)

  input_ids = torch.tensor(padded)
  attention_mask = torch.tensor(attention_mask)

  input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)

  # the padded input ids as well as the attention mask
  with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

  # dimensionality reduction 
  features = last_hidden_states[0][:,0,:].to("cpu").numpy()

  return features


In [6]:
combined = combined[["selftext", "is_stress"]]
combined = combined.rename(columns={"selftext": 0, "is_stress": 1})
combined = combined.dropna()

In [7]:
len(combined)

5978

In [7]:
# calculate the bert_featurs for the dreaddit dataset 
dreaddit = pd.read_csv("data/dreaddit/train.csv")

In [8]:
dreaddit = dreaddit[["text", "label"]]
dreaddit = dreaddit.rename(columns={"text": 0, "label": 1})

In [9]:
len(dreaddit)
# compute bert features for the dreaddit test portion

2838

In [10]:
# create batches of 64
BATCHES = 64
# length of batches 
batch_size = (len(dreaddit) + 1) // BATCHES
# bert_features = np.empty([93, 768])
# bert_features = np.array(bert_features)
bert_features = []

for i in range(BATCHES):
    batch = dreaddit[i*batch_size:(i+1)*batch_size]
    # new_arr = np.append(bert_features, getFeatures(batch), axis=0)
    bert_features.append(getFeatures(batch))
    print(f"iteration {i}")
    # bert_features = new_arr

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg opt

In [11]:
len(bert_features[1])

44

In [12]:
bert_features[1].shape

(44, 768)

In [13]:
features = np.array(bert_features)

In [14]:
features.shape

(64, 44, 768)

In [15]:
# features = features.reshape((64*93), 768)
features = features.reshape((64*44, 768))

In [16]:
features.shape

(2816, 768)

In [17]:
np.savetxt("bert-training-features-dreaddit.csv", features, delimiter=',')